<a href="https://colab.research.google.com/github/sophia-zhang-qwq/animal-pose-est/blob/main/sleap_environment_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<h1></h1>
<h2>SLEAP TRAINING AND INFERENCE GUIDE</h2>
</center>
<center>Delia Curran</center>


<center>April 2, 2025 </center>

# SLEAP Requirements

---
## SLEAP Software Requirements:
*   All training videos must have the same aspect ratio
  *   *see: [https://github.com/talmolab/sleap/issues/2004](https://github.com/talmolab/sleap/issues/2004)*
*   Input scaling must be set to 1
  *   *see: [https://github.com/talmolab/sleap/discussions/925](https://github.com/talmolab/sleap/discussions/925)*
*   Python version must be <=3.10
  *   *see: [https://github.com/talmolab/sleap/issues/2100](https://github.com/talmolab/sleap/issues/2100)*
  *   *guide:[https://gist.github.com/kargaranamir/e0b7910fed0a31](https://gist.github.com/kargaranamir/e0b7910fed0a3189563d9254c7a2c439)*
*   Must use CUDA 11.2 and cuDNN 8.1
  *   *see: [https://github.com/DeepLabCut/DeepLabCut/issues/2465](https://github.com/DeepLabCut/DeepLabCut/issues/2465)*
---

---
## Ongoing Issues:
*   Issues with inference ([https://github.com/talmolab/sleap/issues/628](https://github.com/talmolab/sleap/issues/628))
---

# Configure Training Environment

In [4]:
# override and install python 3.10
! wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
! chmod +x mini.sh
! bash ./mini.sh -b -f -p /usr/local
! conda install -q -y jupyter
! conda install -q -y google-colab -c conda-forge
! python -m ipykernel install --name "py310" --user

--2025-04-24 21:09:24--  https://repo.anaconda.com/miniconda/Miniconda3-py310_22.11.1-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.191.158, 104.16.32.241, 2606:4700::6810:bf9e, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.191.158|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72402405 (69M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  69.05M   102MB/s    in 0.7s    

2025-04-24 21:09:25 (102 MB/s) - ‘mini.sh’ saved [72402405/72402405]

PREFIX=/usr/local
Unpacking payload ...
                                                                               
Installing base environment...





Preparing transaction: - \ | / done
Executing transaction: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpect

# Install SLEAP & Dependencies

In [5]:
# confirm using python 3.10
! python3 --version

Python 3.10.16


In [6]:
# install sleap and dependencies
! pip install sleap[pypi]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 45.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.1/904.1 kB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# install additional sleap dependencies and load GPU support through CUDA
! pip install matplotlib-inline
! pip install ipython
! apt-get install cuda-11-8
! apt-get install -y libcudnn8=8.6.0.163-1+cuda11.8
! pip install numpy==1.23

! export PATH=/usr/local/cuda-11.8/bin:$PATH
! export LD_LIBRARY_PATH=/usr/local/cuda-11.8/lib64:$LD_LIBRARY_PATH
! export CUDNN_INCLUDE_DIR=/usr/local/cuda/include
! export CUDNN_LIB_DIR=/usr/local/cuda/lib64

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cpp-12 cuda-cccl-11-8 cuda-command-line-tools-11-8 cuda-compiler-11-8
  cuda-cudart-11-8 cuda-cudart-dev-11-8 cuda-cuobjdump-11-8 cuda-cupti-11-8
  cuda-cupti-dev-11-8 cuda-cuxxfilt-11-8 cuda-demo-suite-11-8
  cuda-documentation-11-8 cuda-driver-dev-11-8 cuda-drivers cuda-drivers-570
  cuda-gdb-11-8 cuda-libraries-11-8 cuda-libraries-dev-11-8 cuda-memcheck-11-8
  cuda-nsight-11-8 cuda-nsight-compute-11-8 cuda-nsight-systems-11-8
  cuda-nvcc-11-8 cuda-nvdisasm-11-8 cuda-nvml-dev-11-8 cuda-nvprof-11-8
  cuda-nvprune-11-8 cuda-nvrtc-11-8 cuda-nvrtc-dev-11-8 cuda-nvtx-11-8
  cuda-nvvp-11-8 cuda-profiler-api-11-8 cuda-runtime-11-8 cuda-sanitizer-11-8
  cuda-toolkit-11-8 cuda-toolkit-11-8-config-common
  cuda-toolkit-11-config-common cuda-tools-11-8 cuda-visual-tools-11-8
  dctrl-tools default-jre default-jre-headless dkms fakeroot fonts-deja

In [ ]:
# restart runtime so GPU changes take effect
import os
os.kill(os.getpid(), 9)

# Run SLEAP Training

In [1]:
# load sleap training files from google drive
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/sleap/mice_hc")
#! unzip model.slp.training_job.zip
! ls
print("Successfully loaded sleap training files")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
colab.predicted_suggestions.slp  mice_hc.topdown.predictions.slp
inference_times_hist.png	 mice_hc.train.pkg.slp
inference_times_plot.png	 mini.sh
mice_hc.demo.mp4		 models
mice_hc.demo.predictions.slp	 video_output.mp4
Successfully loaded sleap training files


In [2]:
# keep the sleap program running in background
import threading
import time
def keep_colab_alive():
    while True:
        print("Keeping Colab Session Active...")
        time.sleep(600)
threading.Thread(target=keep_colab_alive, daemon=True).start()

Keeping Colab Session Active...


In [4]:
# run top-down sleap training using loaded training job
#! sleap-train centroid.json trained_model.pkg.slp
#! sleap-train centered_instance.json trained_model.pkg.slp

#FileNotFoundError: Could not find training profile: centroid.json
! sleap-train baseline.centroid.json mice_hc.train.pkg.slp --run_name "mice_hc.single.centroid"

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:sleap.nn.training:Versions:
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
INFO:sleap.nn.training:Training labels file: mice_hc.train.pkg.slp
INFO:sleap.nn.training:Training profile: /usr/local/lib/python3.10/site-packages/sleap/training_profiles/baseline.centroid.json
INFO:sleap.nn.training:
INFO:sleap.nn.training:Arguments:
INFO:sleap.nn.training:{
    "training_job_path": "baseline.centroid.json",
    "labels_path": "mice_hc.train.pkg.slp",
    "video_paths": [
        ""
    ],
    "val_labels": null,
    "test_labels": null,
    "base_checkpoint": null,
    "tensorboard": false,
    "save_viz": false,
    "keep_viz": false,
    "zmq": false,
    "publish_port": 9001,
    "controller_port": 9000,
    "run_name": "mice_hc.single.centroid",
    "prefix": "",
    "suffix": "",
    "cpu": false,
    "first_gpu": false,
    "last_gpu": false,
    "gpu": "auto"
}
INFO:sleap.nn.

In [7]:
# run top-down sleap training using loaded training job
#! sleap-train centroid.json trained_model.pkg.slp
#! sleap-train centered_instance.json trained_model.pkg.slp

#FileNotFoundError: Could not find training profile: centered_instance.json
! sleap-train baseline_large_rf.single.json mice_hc.train.pkg.slp --run_name "mice_hc.single.centered_instance"

/bin/bash: line 1: sleap-train: command not found


In [ ]:
# run multi-class top-down-id sleap training using loaded training job (instance tracking)
! sleap-train centroid.json trained_model.pkg.slp
! sleap-train multi_class_topdown.json trained_model.pkg.slp

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Traceback (most recent call last):
  File "/usr/local/bin/sleap-train", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/site-packages/sleap/nn/training.py", line 2038, in main
    trainer = create_trainer_using_cli(args=args)
  File "/usr/local/lib/python3.10/site-packages/sleap/nn/training.py", line 1939, in create_trainer_using_cli
    raise FileNotFoundError(f"Could not find training profile: {job_filename}")
FileNotFoundError: Could not find training profile: centroid.json
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
Traceback (most recent call last):
  File "/usr/local/bin/sleap-train", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/site-packages/sleap/nn/training.py", line 2038, in main
    trainer = create_trainer_using_cli(args=args)
  File "/usr/local/lib/python3.10/site-packages/sleap/nn/training.py", line 1939, in create_trainer_using_cli
    raise FileNotFoundEr

# Run SLEAP Inference

In [ ]:
# load sleap inference files from google drive
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/sleap/models"
!ls "/content/drive/My Drive/sleap"
print("Successfully loaded sleap inference files")

In [ ]:
# predicting instances in suggested frames from exported .slp file
!sleap-track \
    -m /content/drive/My\ Drive/sleap/models/top-down250401_145841.centroid \
    -m /content/drive/My\ Drive/sleap/models/top-down250401_145841.centered_instance \
    --only-suggested-frames \
    --threshold 0.05 \
    -o 001.predictions.slp \
    /content/drive/My\ Drive/sleap/trained_model.pkg.slp

# if running top-down-id
# -m /content/drive/My\ Drive/Academic/sleap/models/top-down250401_145841.multi_class_topdown \

In [ ]:
# predicting and tracking instances in uploaded video (example.mp4)
! ffmpeg -i 'example.mp4' -vf "scale=1280:720:force_original_aspect_ratio=decrease,pad=1280:720:(ow-iw)/2:(oh-ih)/2:black" 'crop_example.mp4'
!sleap-track example.mp4\
    --frames 0-100 \
    --tracking.tracker simple \
    -m /content/drive/My\ Drive/sleap/models/001250328_201355.centered_instance \
    -m /content/drive/My\ Drive/sleap/models/001250328_201355.centroid

# Testing

In [3]:
!python -c "import sleap; sleap.disable_preallocation(); print(sleap.versions()); sleap.system_summary()"

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
2025-04-24 19:56:41.553786: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-24 19:56:41.628519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-04-24 19:56:41.628869: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
SLEAP: 1.4.1
TensorFlow: 2.8.4
Numpy: 1.23.0
Python: 3.10.16
OS: Linux-6.1.123+-x86_64-with-glibc2.35
None
GPUs: 1/1 available
  Device: /physical_device:GPU:0
         Available: True
       Initialized: False
     Memory growth: True
